In [50]:
import pandas as pd

# Đọc dữ liệu từ URL
url = 'https://docs.google.com/spreadsheets/d/1zIXxEL9WoVnjl_LKj-22Zkexq55xQ1PWU3bKOgG8s2I/export?format=csv&gid=436867342'

data = pd.read_csv(url)

In [ ]:
data.shape

(72314, 17)

In [ ]:
data.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72314 entries, 0 to 72313
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Date                               72314 non-null  object
 1   Time customer placed order         72314 non-null  object
 2   Time order placed at restaurant    72154 non-null  object
 3   Time driver arrived at restaurant  54190 non-null  object
 4   Delivery time                      72314 non-null  object
 5   Driver ID                          72314 non-null  int64 
 6   Driver Name                        72314 non-null  object
 7   Restaurant ID                      72314 non-null  int64 
 8   Customer ID                        72314 non-null  int64 
 9   Delivery Area                      72314 non-null  object
 10  ASAP                               72314 non-null  object
 11  Sub Total                          72314 non-null  object
 12  Deli

In [ ]:
data.duplicated().sum()

0

In [ ]:
print(data.isnull().sum())

Date                                     0
Time customer placed order               0
Time order placed at restaurant        160
Time driver arrived at restaurant    18124
Delivery time                            0
Driver ID                                0
Driver Name                              0
Restaurant ID                            0
Customer ID                              0
Delivery Area                            0
ASAP                                     0
Sub Total                                0
Delivery fee                             0
Service fee                              0
Discount                                 0
Tip                                      0
Refunded amount                          0
dtype: int64


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Driver ID,72314.0,9.221850e+03,124.060497,9007.0,9115.0,9222.0,9329.0,9438.0
Restaurant ID,72314.0,7.205541e+03,117.798753,7002.0,7104.0,7204.0,7307.0,7409.0
Customer ID,72314.0,1.100286e+06,57847.182260,1000052.0,1050173.0,1101445.0,1149893.0,1200360.0


In [51]:
# 1. Đổi tên các column có name chứa khoảng cách thành không có khoảng cách
data.columns = [col.replace(' ', '_') for col in data.columns]

In [52]:
# 2. Xóa cột Time_driver_arrived_at_restaurant và ASAP
columns_to_drop = ['Time_driver_arrived_at_restaurant', 'ASAP']
data.drop(columns=[col for col in columns_to_drop if col in data.columns], inplace=True)


In [53]:
# Xử lí các giá trị Null của cột 'Time_order_placed_at_restaurant'
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')


In [54]:
# Chuyển các cột thời gian sang datetime kết hợp với Date
data['Time_customer_placed_order'] = pd.to_datetime(
    data['Date'].dt.strftime('%Y-%m-%d') + ' ' + data['Time_customer_placed_order'],
    format='%Y-%m-%d %H:%M:%S'
)
data['Time_order_placed_at_restaurant'] = pd.to_datetime(
    data['Date'].dt.strftime('%Y-%m-%d') + ' ' + data['Time_order_placed_at_restaurant'],
    format='%Y-%m-%d %H:%M:%S'
)
data['Delivery_time'] = pd.to_datetime(
    data['Date'].dt.strftime('%Y-%m-%d') + ' ' + data['Delivery_time'],
    format='%Y-%m-%d %H:%M:%S'
)

# Xử lý thời gian qua ngày mới
data.loc[data['Time_order_placed_at_restaurant'] < data['Time_customer_placed_order'], 'Time_order_placed_at_restaurant'] += pd.Timedelta(days=1)
data.loc[data['Delivery_time'] < data['Time_customer_placed_order'], 'Delivery_time'] += pd.Timedelta(days=1)


In [60]:
# Tính trung bình của (Delivery_time - Time_order_placed_at_restaurant)
valid_time_diff = data['Delivery_time'] - data['Time_order_placed_at_restaurant']
average_time_diff = valid_time_diff[~valid_time_diff.isna()].mean()
# Làm tròn giây của average_time_diff
average_time_diff = average_time_diff.round('S')

<ipython-input-60-64695ac863f7>:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  average_time_diff = average_time_diff.round('S')


In [61]:
print("Average time difference:", average_time_diff)

Average time difference: 0 days 01:24:18


In [72]:
# Điền giá trị Null trong 'Time_order_placed_at_restaurant'
data['Time_order_placed_at_restaurant'] = data.apply(
    lambda row: (row['Delivery_time'] - average_time_diff).round('S')
    if pd.isna(row['Time_order_placed_at_restaurant'])
    else row['Time_order_placed_at_restaurant'].round('S'),
    axis=1)


<ipython-input-72-d5b52a17f85d>:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  else row['Time_order_placed_at_restaurant'].round('S'),


In [73]:
# Định dạng lại cột Date chỉ lấy ngày
data['Date'] = pd.to_datetime(data['Date']).dt.date

In [74]:
#Tạo table Driver (gồm Driver_ID với Driver_Name)
if 'Driver_ID' in data.columns and 'Driver_Name' in data.columns:
    driver_table = data[['Driver_ID', 'Driver_Name']].drop_duplicates()

In [75]:
#Tạo table Restaurant (gồm Restaurant_ID với Delivery_Area)
if 'Restaurant_ID' in data.columns and 'Delivery_Area' in data.columns:
    restaurant_table = data[['Restaurant_ID', 'Delivery_Area']].drop_duplicates()

In [77]:
# Lưu dữ liệu
data.to_csv('Cleaned_Data.csv', index=False)

In [78]:
driver_table.to_csv('driver_table.csv', index=False)
restaurant_table.to_csv('restaurant_table.csv', index=False)